In [1]:
# Import base packages
import tiledb
import tiledb.cloud
from tiledb.cloud.compute import Delayed, DelayedSQL, DelayedArrayUDF
import numpy

In [2]:
x = Delayed(numpy.median)

In [3]:
# You can see the type is now `Delayed`
x

In [15]:
# A few base functions:
import random
node_1 = Delayed(numpy.median, local=True, name="node_1")([1, 2, 3])
node_2 = Delayed(lambda x: x * 2, local=True, name="node_2")(node_1)
node_3 = Delayed(lambda x: x * 2, local=True, name="node_3")(node_2)

nodes_by_name= {'node_1': node_1, 'node_2': node_2, 'node_3': node_3}
#Function which sleeps for some time so we can see the graph in different states
def f():
    import time
    import random
    x = random.randrange(0, 30)
    time.sleep(x)
    return x

# Randomly add 96 other nodes to the graph. All of these will use the sleep function
for i in range(4, 600):
    name = "node_{}".format(i)
    node = Delayed(f, local=True, name=name)()
    
    dep = random.randrange(1, i-1)
    # Randomly set dependency on one other node
    node_dep = nodes_by_name["node_{}".format(dep)]
    # Force the dependency to be set
    node.depends_on(node_dep)
    
    nodes_by_name[name] = node


In [16]:
node_1.visualize()

Visualize(value='{"nodes": ["9468ce91-d15b-4147-874a-ea828578a1c8", "6d667b8f-fccf-4eb1-9a4c-ac408906dfae", "e…

In [9]:
node_99 = nodes_by_name["node_99"]
node_99.compute()

10